In [ ]:
!pip install streamlit pyngrok ultralytics opencv-python-headless -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 138.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 147.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image
import cv2
import numpy as np

# Load the YOLOv8n detection model (update the path to your model file)
model = YOLO('best.pt')  # Replace with the path to your trained model

# Streamlit app layout
st.title("Material Detection with YOLOv8n")
st.write("Upload an image to detect Alucan, Glass, HDPEM, or PET with bounding boxes.")

# File uploader for image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Read the image
    image = Image.open(uploaded_file)
    image_np = np.array(image)
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # Display the uploaded image
    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("Detecting...")

    # Perform prediction
    results = model.predict(image_cv, conf=0.5)  # Adjust confidence threshold as needed

    # Define class names
    class_names = ['Alucan', 'Glass', 'HDPEM', 'PET']  # Your class names

    # Check for detections
    if results[0].boxes is not None and len(results[0].boxes) > 0:
        # Draw bounding boxes and labels on the image
        for box in results[0].boxes:
            # Get box coordinates, class, and confidence
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            class_idx = int(box.cls)  # Class index
            conf = box.conf.item()  # Confidence score
            label = f"{class_names[class_idx]}: {conf:.2f}"

            # Draw rectangle and label
            cv2.rectangle(image_cv, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            cv2.putText(image_cv, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Convert back to RGB for display
        image_annotated = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)

        # Display the annotated image
        st.image(image_annotated, caption='Detected Objects', use_column_width=True)

        # List detected objects
        st.write("**Detected Objects**:")
        for box in results[0].boxes:
            class_idx = int(box.cls)
            conf = box.conf.item()
            st.write(f"- {class_names[class_idx]} (Confidence: {conf:.2f})")
    else:
        st.error("No objects detected in the image.")

Writing app.py


In [ ]:
from pyngrok import ngrok

# Set ngrok authtoken (replace with your actual authtoken)
!ngrok authtoken "2w082XSHzJcB0qdg9zDKWdeRh7p_6ak8T4bUgodbqWyZ62vCE"  # Replace with your ngrok authtoken

# Terminate any existing ngrok tunnels
ngrok.kill()

# Start a new ngrok tunnel to the Streamlit port (8501)
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run app.py --server.port 8501

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://a3cd-34-125-218-93.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.218.93:8501

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2025-05-21 11:14:35.029 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of

In [ ]:
from ultralytics import YOLO
model = YOLO('best.pt')
print(model.task)

detect
